In [1]:
import sys
import os
import numpy as np
import pandas as pd
from PIL import Image

import ipywidgets as widgets
from io import BytesIO

from ganpfinder import GANPfinder
from jupyterGUI import GUIses

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


### Description:

Possible datasets: ffhq, car, cat, church, horse, bedrooms, kitchen, places

---

In [3]:
gpf = GANPfinder(class_name="car", gan_sample_seed=0, n_comp_in_use=5)
guises = GUIses(gpf)
guises.run()

In [ ]:
# CONTINUE

---